In [ ]:
pip install requests

In [ ]:
pip install biopython

In [ ]:
pip install odfpy

In [ ]:
!wget -O "openTECR recuration.ods" "https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=ods"

--2025-01-19 09:11:24--  https://docs.google.com/spreadsheets/d/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c/export?format=ods
Resolving docs.google.com (docs.google.com)... 142.251.179.100, 142.251.179.138, 142.251.179.101, ...
Connecting to docs.google.com (docs.google.com)|142.251.179.100|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-00-8k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/q2h8j528ksl67k4dceiprkoqoc/1737277880000/115120384215235060766/*/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c?format=ods [following]
--2025-01-19 09:11:24--  https://doc-00-8k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/q2h8j528ksl67k4dceiprkoqoc/1737277880000/115120384215235060766/*/1jLIxEXVzE2SAzIB0UxBfcFoHrzjzf9euB6ART2VDE8c?format=ods
Resolving doc-00-8k-sheets.googleusercontent.com (doc-00-8k-sheets.googleusercontent.com)... 172.253.115.132, 2607:f8b0:4004:c06::84
Connecting to doc-00-8k-sheets.googleuser

In [ ]:
import pandas as pd
from Bio import Entrez
import numpy as np
import time

def crossref(doi):
    url = f"https://api.crossref.org/works/{doi}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"DOI not found in CrossRef: {doi}"}

def pubmed(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid, retmode="xml")
    response = Entrez.read(handle)
    handle.close()
    return response

def doires2meta(response):
  metadata = {}
  metadata['Title'] = response['message']['title']
  date = response['message']['created']['date-parts'][0]
  metadata['Date'] = f'{date[2]}.{date[1]}.{date[0]}'
  metadata['Publisher'] = response['message']['publisher']
  metadata['License'] = response['message']['license'][0]['URL']
  metadata['Type'] = response['message']['type']
  metadata['Volume'] = response['message']['volume']
  metadata['Issue'] = response['message']['issue']
  metadata['Page'] = response['message']['page']
  fn = []
  sn = []
  for b in response['message']['author']:
    fn.append(b['given'])
    sn.append(b['family'])

  string = ''

  for i in range(len(fn)):
    string = string + fn[i] + " " + sn[i] + ", "

  string = string[:-2]
  metadata['Authors'] = string
  metadata['Language'] = response['message']['language']
  return metadata

def pubmed2meta(response):
  metadata = {}
  try:
    metadata['Language'] = response['PubmedArticle'][0]['MedlineCitation']['Article']['Language']
  except:
    metadata['Language'] = '-'
  try:
    metadata['Volume'] = response['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['Volume']
  except:
    metadata['Volume'] = '-'
  try:
    metadata['Issue'] = response['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['Issue']
  except:
    metadata['Issue'] = "-"
  try:
    metadata['Page'] = response['PubmedArticle'][0]['MedlineCitation']['Article']['Pagination']['MedlinePgn']
  except:
    metadata['Page'] = "-"
  date  = response['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
  try:
    metadata['Journal'] = response['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['Title']
  except:
    metadata['Journal'] = "-"
  try:
    metadata['Abstract'] = response['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
  except:
    metadata['Abstract'] = '-'
  #print(date)
  try:
    month = date['Month']
  except:
    month = ' '
  year = date['Year']
  metadata['Date'] = f'{month} {year}'
  try:
    fn = []
    sn = []
    for b in response['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']:
      fn.append(b['ForeName'])
      sn.append(b['LastName'])

    string = ''

    for i in range(len(fn)):
      string = string + fn[i] + " " + sn[i] + ", "

    string = string[:-2]
    metadata['Authors'] = string
  except:
    metadata['Authors'] = "-"
  return metadata

def return_blank(indi):
  metadata = {}
  if indi == "pub":
    metadata['Language'] = '-'
    metadata['Volume'] = '-'
    metadata['Issue'] = '-'
    metadata['Page'] = '-'
    metadata['Journal'] = '-'
    metadata['Abstract'] = '-'
    metadata['Date'] = '-'
    metadata['Authors'] = '-'
  if indi == "doi":
    metadata['Title'] = '-'
    metadata['Language'] = '-'
    metadata['Volume'] = '-'
    metadata['Issue'] = '-'
    metadata['Page'] = '-'
    metadata['Publisher'] = '-'
    metadata['Date'] = '-'
    metadata['Authors'] = '-'
    metadata['License'] = '-'
    metadata['Type'] = '-'
  return metadata

In [ ]:
Entrez.email = "ruslanibragimovut@outlook.com"
df = pd.read_excel("openTECR recuration.ods", sheet_name="references")
pmids = df['pmid'].to_list()
dois = df['doi'].to_list()


In [ ]:
doismeta = []
for d in dois:
  if type(d) == type('Is a string?'):
    r = crossref(d)
    #print(r)

    try:
      d = doires2meta(r)
    except:
      d = return_blank('doi')

    doismeta.append(d)
  else:
    c = return_blank('doi')
    doismeta.append(c)

In [ ]:
pmidsmeta = []
for pmid in pmids:
  if np.isnan(pmid) == False:
    time.sleep(1)
    r = pubmed(str(pmid))
    #print(r)
    d = pubmed2meta(r)
    pmidsmeta.append(d)
  else:
    c = return_blank('pub')
    pmidsmeta.append(c)

In [ ]:
dfpub = pd.DataFrame(pmidsmeta)
dfdoi = pd.DataFrame(doismeta)

In [ ]:
dfpubC = pd.concat([df, dfpub], axis=1)
dfdoiC = pd.concat([df, dfdoi], axis=1)
dfpubC.to_csv('openTECRmetadataPubMed.csv')
dfdoiC.to_csv('openTECRmetadataDOI.csv')